<a href="https://colab.research.google.com/github/Ahmad20/Hate-Speech-NLP/blob/main/Proyek_Pertama_Membuat_Model_NLP_dengan_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Get Data

In [3]:
#Library panda
import pandas as pd
#Mendapatkan data
df = pd.read_csv("https://raw.githubusercontent.com/Ahmad20/Hate-Speech-NLP/main/labeled_data.csv")
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [5]:
#@markdown Dataset yang akan dipakai bebas, namun minimal memiliki 1000 sampel.
df.shape

(24783, 7)

#Data Split

In [6]:
#Hanya menggunakan kolom tweet dan class
df = df[['tweet','class']]
#Mendapatkan data dummy dari kolom class
label=pd.get_dummies(df['class'])
#Drop kolom class karena sudah memiliki dummy==>tersisa kolom tweet
df=df.drop('class',axis=1,inplace=False)
#Menggabungkan kolom tweet dan data dummy class
df = pd.concat([df,label],axis=1)
#Mendapatkan value dari kolom tweet berupa array
tweet = df['tweet'].values
#Mendapatkan value dari kolom-kolom dummy class berupa array
cat = df.drop('tweet',axis=1, inplace=False).values

#Train Test Split

In [7]:
#@markdown Validation set sebesar 20% dari total dataset.
from sklearn.model_selection import train_test_split
#Membagi data test 20% dan data train 80%
tweet_latih, tweet_test, cat_latih, cat_test=train_test_split(
    tweet, cat, test_size=0.2)

#Tokenizer Sentences

In [8]:
#@markdown Harus menggunakan fungsi tokenizer.
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
#Membatasi jumlah kata menjadi 5000 dan pengganti
# kata yang tidak termasuk list menjadi x
tokenizer = Tokenizer(num_words=5000, oov_token='x')
#Mengaplikasikan token pada tweet_latih dan tweet_test
tokenizer.fit_on_texts(tweet_latih)
tokenizer.fit_on_texts(tweet_test)
#Convert text menjadi sequence
sekuens_latih = tokenizer.texts_to_sequences(tweet_latih)
sekuens_test = tokenizer.texts_to_sequences(tweet_test)
#Convert sequence menjadi 2D Numpy array dengan pad_sequences
padded_latih = pad_sequences(sekuens_latih)
padded_test = pad_sequences(sekuens_test)

#Model

In [9]:
#@markdown Harus menggunakan model sequential.\
#@markdown Harus menggunakan Embedding.\
#@markdown Harus menggunakan LSTM dalam arsitektur model.
#Library TensorFlow
import tensorflow as tfw
#Assign model
model = tfw.keras.Sequential([
                             tfw.keras.layers.Embedding(input_dim=5000, 
                                                        output_dim=16),
                             tfw.keras.layers.LSTM(64),
                             tfw.keras.layers.Dense(128, activation="relu"),
                             tfw.keras.layers.Dense(64, activation="relu"),
                             tfw.keras.layers.Dense(3, activation="softmax")
])
#Compile Model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
#@markdown Akurasi dari model minimal 75%.
#Jumlah loop model
num_epochs = 40
#Menjalankan model
history = model.fit(padded_latih, cat_latih, epochs=num_epochs,
                    validation_data=(padded_test, cat_test),verbose=2)

Epoch 1/40
620/620 - 30s - loss: 0.4389 - accuracy: 0.8495 - val_loss: 0.3294 - val_accuracy: 0.8915
Epoch 2/40
620/620 - 27s - loss: 0.2631 - accuracy: 0.9124 - val_loss: 0.2733 - val_accuracy: 0.9007
Epoch 3/40
620/620 - 27s - loss: 0.2025 - accuracy: 0.9299 - val_loss: 0.3137 - val_accuracy: 0.8933
Epoch 4/40
620/620 - 27s - loss: 0.1707 - accuracy: 0.9411 - val_loss: 0.3582 - val_accuracy: 0.8961
Epoch 5/40
620/620 - 27s - loss: 0.1435 - accuracy: 0.9486 - val_loss: 0.3736 - val_accuracy: 0.8788
Epoch 6/40
620/620 - 27s - loss: 0.1223 - accuracy: 0.9574 - val_loss: 0.4290 - val_accuracy: 0.8870
Epoch 7/40
620/620 - 28s - loss: 0.1058 - accuracy: 0.9629 - val_loss: 0.4435 - val_accuracy: 0.8780
Epoch 8/40
620/620 - 28s - loss: 0.0915 - accuracy: 0.9674 - val_loss: 0.4632 - val_accuracy: 0.8830
Epoch 9/40
620/620 - 28s - loss: 0.0791 - accuracy: 0.9736 - val_loss: 0.5976 - val_accuracy: 0.8669
Epoch 10/40
620/620 - 27s - loss: 0.0659 - accuracy: 0.9772 - val_loss: 0.5911 - val_accura